In [2]:
import requests
import base64
import json
import time
from PIL import Image
from io import BytesIO
import os
from dotenv import load_dotenv 

# --- 1. Configuration ---
load_dotenv()

api_key = os.getenv("RUNPOD_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please create a .env file and add RUNPOD_API_KEY=your_key")

workflow_file_name = "LoadAudio-VibeVoiceSS-Maya=v2.json"
endpoint_id = "sz4vfpd14gvjyo"

# --- 2. Define the API URLs ---
run_url = f"https://api.runpod.ai/v2/{endpoint_id}/run"
status_url_template = f"https://api.runpod.ai/v2/{endpoint_id}/status/"

# --- 3. Load the Workflow from the JSON File ---
print(f"Loading workflow from '{workflow_file_name}'...")
with open(workflow_file_name, 'r') as f:
    workflow_from_file = json.load(f)

workflow = {
    "input": {
        "workflow": workflow_from_file
    }
}

# --- 4. Send the Initial API Request to Start the Job ---
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

print("Sending request to start the job...")
response = requests.post(run_url, headers=headers, json=workflow)

if response.status_code == 200:
    response_data = response.json()
    job_id = response_data.get('id')
    
    if not job_id:
        print("Error: API response did not include a job ID.")
        print("Full response:", json.dumps(response_data, indent=2))
    else:
        print(f"Job successfully started with ID: {job_id}")

        # --- 5. Poll for the Job Status ---
        status_url = status_url_template + job_id
        while True:
            print("Polling for job status...")
            status_response = requests.get(status_url, headers=headers)
            status_data = status_response.json()
            job_status = status_data.get('status')

            if job_status == 'COMPLETED':
                print("Job completed successfully!")
                
                # --- 6. Process the Final Output ---
                try:
                    output = status_data.get('output', {})
                    
                    # Handle audio outputs
                    if 'audio' in output and output['audio']:
                        print("Processing audio output...")
                        for audio_item in output['audio']:
                            audio_base64 = audio_item['data']
                            filename = audio_item['filename']
                            
                            # Decode and save the audio file
                            audio_bytes = base64.b64decode(audio_base64)
                            with open(filename, 'wb') as f:
                                f.write(audio_bytes)
                            
                            print(f"Audio successfully generated and saved as '{filename}'")
                    
                    # Handle image outputs (for workflows that generate images)
                    elif 'images' in output and output['images']:
                        print("Processing image output...")
                        for image_item in output['images']:
                            image_base64 = image_item['data'] 
                            filename = image_item['filename']
                            
                            image_bytes = base64.b64decode(image_base64)
                            image = Image.open(BytesIO(image_bytes))
                            
                            image.save(filename)
                            print(f"Image successfully generated and saved as '{filename}'")
                    
                    else:
                        print("No recognized output format (images or audio) found in response")
                        print("Available output keys:", list(output.keys()))

                except Exception as e:
                    print(f"\n Error: Could not parse output data from the final response: {e}\n")
                    
                    # Debug output with redacted base64 data
                    debug_data = json.loads(json.dumps(status_data))
                    try:
                        # Redact both image and audio data for debugging
                        if 'output' in debug_data:
                            if 'images' in debug_data['output']:
                                for img in debug_data['output']['images']:
                                    if 'data' in img:
                                        original_length = len(img['data'])
                                        img['data'] = f"<base64 data redacted, original length: {original_length}>"
                            
                            if 'audio' in debug_data['output']:
                                for audio in debug_data['output']['audio']:
                                    if 'data' in audio:
                                        original_length = len(audio['data'])
                                        audio['data'] = f"<base64 data redacted, original length: {original_length}>"
                    except Exception:
                        pass 
                        
                    print("Full response (redacted):", json.dumps(debug_data, indent=2))
                
                break

            elif job_status in ['IN_QUEUE', 'IN_PROGRESS']:
                time.sleep(5)
            else:
                print(f"Job execution failed or was cancelled.")
                print("Final Status:", job_status)
                if 'output' in status_data:
                    print("Output/Error:", status_data['output'])
                else:
                    print("Full response:", json.dumps(status_data, indent=2))
                break 

else:
    print(f"Error: Initial API request failed with status code {response.status_code}")
    print("Response:", response.text)

Loading workflow from 'LoadAudio-VibeVoiceSS-Maya=v2.json'...
Sending request to start the job...
Job successfully started with ID: 9003cd4a-2f9a-4bfb-99b7-35750fa3f160-u1
Polling for job status...
Polling for job status...
Job completed successfully!
Processing audio output...
Audio successfully generated and saved as 'ComfyUI_00001_.flac'
